# TENG Gait Analysis: ML Pipeline

This notebook covers the complete machine learning pipeline for the TENG gait sensor project, from data loading to model training and evaluation.

**Goals:**
1.  **Activity Classification:** (4 classes: stand, walk, run, jump)
2.  **Clinical QOM Assessment:** (3 classes: Intervention Priority, Improvement Recommended, Healthy Baseline)
3.  **Bio-authentication:** (Regression: Predict weight)
4.  **Clinical Metrics:** (Activity Duration, QOM Confidence)

## 1. Setup & Data Loading

This section handles library imports and loading the dataset.

**For Google Colab:**
1.  Upload your `User_Gait_Data_Master.zip` to your Google Drive.
2.  Run the first cell and authorize Google Drive.
3.  The second cell will unzip the data into your Colab environment.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from pathlib import Path

from sklearn.model_selection import train_test_split, cross_val_score, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import classification_report, confusion_matrix, mean_absolute_error, r2_score
from sklearn.exceptions import DataConversionWarning
import warnings

# Suppress common warnings
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)

# Check if running in Google Colab
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Not running in Google Colab. Assuming local setup.")

In [ ]:
# --- Define Data Paths ---
# Base path for the data
if IN_COLAB:
    # Unzip the data from Google Drive to the Colab environment
    print("Unzipping data from Google Drive...")
    !unzip -q -o "/content/drive/My Drive/User_Gait_Data_Master.zip" -d "/content/"
    DATA_DIR = Path("/content/User_Gait_Data_Master")
else:
    # Local path
    DATA_DIR = Path("User_Gait_Data_Master")

SIGNALS_DIR = DATA_DIR / "User_Data_Labelled"
LABELS_FILE = DATA_DIR / "QOM" / "gait_labels_qom.csv"

print(f"Data directory set to: {DATA_DIR}")
print(f"Signals directory set to: {SIGNALS_DIR}")
print(f"Labels file set to: {LABELS_FILE}")

In [ ]:
# Load the master labels file
try:
    labels_df = pd.read_csv(LABELS_FILE)
    print(f"Successfully loaded labels file with {len(labels_df)} rows.")
except FileNotFoundError:
    print(f"ERROR: Labels file not found at {LABELS_FILE}")
    print("Please check your file path and ensure you have unzipped the data.")

labels_df.head()

## 2. Data Visualization (GUI Equivalent)

As requested, here is a simple visualization function to plot any signal from the dataset. This acts as a simple 'GUI' to explore the data.

In [ ]:
def plot_signal_by_name(file_name):
    """Loads and plots a signal CSV given its file_name from the labels_df."""
    
    try:
        # Find the full path and metadata
        metadata = labels_df[labels_df['file_path'] == file_name].iloc[0]
        file_path = SIGNALS_DIR / file_name
        
        # Load the signal data
        signal_df = pd.read_csv(file_path)
        
        # Get activity duration
        duration = signal_df['Time(s)'].max() - signal_df['Time(s)'].min()
        
        # Create the plot
        plt.figure(figsize=(15, 5))
        plt.plot(signal_df['Time(s)'], signal_df['Voltage(V)'])
        plt.title(f"Signal for: {file_name} (Activity: {metadata['activity']})")
        plt.xlabel("Time (s)")
        plt.ylabel("Voltage (V)")
        plt.text(0.01, 0.9, f"Subject: {metadata['subject_id']}", transform=plt.gca().transAxes)
        plt.text(0.01, 0.8, f"QOM: {metadata['qom']}", transform=plt.gca().transAxes)
        plt.text(0.01, 0.7, f"Duration: {duration:.2f}s", transform=plt.gca().transAxes)
        plt.grid(True, linestyle=':')
        plt.show()
        
    except FileNotFoundError:
        print(f"ERROR: Signal file not found at {file_path}")
    except IndexError:
        print(f"ERROR: File name '{file_name}' not found in labels file.")
    except Exception as e:
        print(f"An error occurred: {e}")

# --- Example Plots ---
print("Plotting example signals for each activity...")

# Find one example for each activity
plot_signal_by_name(labels_df[labels_df['activity'] == 'stand']['file_path'].iloc[0])
plot_signal_by_name(labels_df[labels_df['activity'] == 'walk']['file_path'].iloc[0])
plot_signal_by_name(labels_df[labels_df['activity'] == 'run']['file_path'].iloc[0])
plot_signal_by_name(labels_df[labels_df['activity'] == 'jump']['file_path'].iloc[0])

## 3. Feature Engineering

This is the most critical step. We will read every CSV file and extract a set of statistical features to represent the *entire* signal with a *single* row of data. This is what the models will be trained on.

In [ ]:
def extract_features(file_path):
    """Extracts a set of features from a single signal CSV file."""
    try:
        df = pd.read_csv(file_path)
        v = df['Voltage(V)']
        t = df['Time(s)']
        
        if v.empty:
            return None
        
        features = {
            'v_mean': v.mean(),
            'v_std': v.std(),
            'v_max': v.max(),
            'v_min': v.min(),
            'v_range': v.max() - v.min(),
            'v_abs_mean': v.abs().mean(),
            'v_skew': v.skew(),
            'v_kurt': v.kurt(),
            'duration': t.max() - t.min()
            # Add more features here (e.g., from scipy.stats)
        }
        return features
        
    except pd.errors.EmptyDataError:
        return None
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

In [ ]:
# Loop over all files in the labels_df and apply feature extraction
print("Starting feature extraction for 84 files...")

all_features = []
for file_name in labels_df['file_path']:
    file_path = SIGNALS_DIR / file_name
    features = extract_features(file_path)
    
    if features:
        all_features.append(features)
    else:
        print(f"Warning: Could not process {file_name}")
        all_features.append({}) # Append empty dict to maintain index

# Create a DataFrame from the features
features_df = pd.DataFrame(all_features)

# Combine features with labels
# We reset index to ensure a clean join
ml_df = pd.concat([labels_df.reset_index(drop=True), features_df.reset_index(drop=True)], axis=1)

# Add BMI
ml_df['bmi'] = ml_df['weight_kg'] / ((ml_df['height_cm'] / 100) ** 2)

# Map QOM to 3-class system
def map_qom(qom_score):
    if qom_score <= 3:
        return 'Intervention Priority' # Class 0
    elif qom_score <= 6:
        return 'Improvement Recommended' # Class 1
    else:
        return 'Healthy Baseline' # Class 2

ml_df['qom_3_class'] = ml_df['qom'].apply(map_qom)

print(f"Feature extraction complete. Created DataFrame with shape: {ml_df.shape}")
ml_df.head()

## 4. Model Training: Activity Classification (4-Class)

**Goal:** Predict `activity` (stand, walk, run, jump) from sensor features.

In [ ]:
print("--- Model 1: Activity Classification (4-Class) ---")

# 1. Define Features (X) and Target (y)
# We include demographic features as they might be relevant (e.g., weight affects jump signal)
feature_cols = ['v_mean', 'v_std', 'v_max', 'v_min', 'v_range', 'v_abs_mean', 
                'v_skew', 'v_kurt', 'duration', 'bmi']
target_col = 'activity'

# Handle missing data (e.g., from failed feature extraction or missing BMI)
model_data = ml_df[feature_cols + [target_col, 'subject_id']].dropna()

X = model_data[feature_cols]
y = model_data[target_col]

# Encode target labels (e.g., 'walk' -> 1)
le_activity = LabelEncoder()
y_encoded = le_activity.fit_transform(y)
activity_classes = le_activity.classes_

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2. Train/Test Split (Using GroupShuffleSplit to keep subjects separate)
# This ensures data from one subject isn't in both train and test sets.
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X_scaled, y_encoded, groups=model_data['subject_id']))

X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]

print(f"Data split: {len(y_train)} train, {len(y_test)} test samples.")
print(f"Test subjects: {np.unique(model_data['subject_id'].iloc[test_idx])}")

# 3. Train Base Model: Random Forest
print("\nTraining Random Forest...")
rf_activity = RandomForestClassifier(n_estimators=100, random_state=42)
rf_activity.fit(X_train, y_train)
y_pred_rf = rf_activity.predict(X_test)

print("Random Forest Results:")
print(classification_report(y_test, y_pred_rf, target_names=activity_classes))

# 4. Train Advanced Model: XGBoost
print("\nTraining XGBoost...")
xgb_activity = XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, eval_metric='mlogloss')
xgb_activity.fit(X_train, y_train)
y_pred_xgb = xgb_activity.predict(X_test)

print("XGBoost Results:")
print(classification_report(y_test, y_pred_xgb, target_names=activity_classes))

# 5. Plot Confusion Matrix (for better model)
cm = confusion_matrix(y_test, y_pred_xgb, normalize='true')
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='.2f', cmap='Blues', xticklabels=activity_classes, yticklabels=activity_classes)
plt.title("Activity Classification Confusion Matrix (XGBoost)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

## 5. Model Training: Clinical QOM Assessment (3-Class)

**Goal:** Predict `qom_3_class` (Intervention Priority, Improvement Recommended, Healthy Baseline).

In [ ]:
print("--- Model 2: QOM Assessment (3-Class) ---")

# 1. Define Features (X) and Target (y)
# We will only use 'walk', 'run', and 'jump' data, as 'stand' has no QOM.
qom_data = ml_df[ml_df['activity'].isin(['walk', 'run', 'jump'])]

feature_cols = ['v_mean', 'v_std', 'v_max', 'v_min', 'v_range', 'v_abs_mean', 
                'v_skew', 'v_kurt', 'duration', 'bmi']
target_col = 'qom_3_class'

model_data = qom_data[feature_cols + [target_col, 'subject_id']].dropna()

X = model_data[feature_cols]
y = model_data[target_col]

# Encode target labels
le_qom = LabelEncoder()
y_encoded = le_qom.fit_transform(y)
qom_classes = le_qom.classes_

# Scale features
scaler_qom = StandardScaler()
X_scaled = scaler_qom.fit_transform(X)

# 2. Train/Test Split (GroupShuffleSplit)
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X_scaled, y_encoded, groups=model_data['subject_id']))

X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]

print(f"Data split: {len(y_train)} train, {len(y_test)} test samples.")
print(f"Test subjects: {np.unique(model_data['subject_id'].iloc[test_idx])}")

# 3. Train Model: XGBoost (better for this task)
print("\nTraining XGBoost for QOM...")
xgb_qom = XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, eval_metric='mlogloss')
xgb_qom.fit(X_train, y_train)
y_pred_xgb = xgb_qom.predict(X_test)

print("XGBoost QOM Results:")
print(classification_report(y_test, y_pred_xgb, target_names=qom_classes))

# 4. Clinical Confidence Rating
print("\n--- Clinical Confidence Rating ---")
y_pred_proba = xgb_qom.predict_proba(X_test)

# Show confidence for the first 5 test samples
for i in range(min(5, len(y_test))):
    true_label = qom_classes[y_test[i]]
    pred_label = qom_classes[y_pred_xgb[i]]
    confidence = y_pred_proba[i].max() * 100
    
    print(f"Sample {i}: True='{true_label}', Predicted='{pred_label}', Confidence={confidence:.2f}%")

# 5. Plot Confusion Matrix
cm = confusion_matrix(y_test, y_pred_xgb, normalize='true')
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='.2f', cmap='Greens', xticklabels=qom_classes, yticklabels=qom_classes)
plt.title("Clinical QOM Classification Confusion Matrix (XGBoost)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

## 6. Model Training: Bio-Authentication (Weight Regression)

**Goal:** Predict `weight_kg` (a continuous value) from sensor features.

In [ ]:
print("--- Model 3: Bio-Authentication (Weight Regression) ---")

# 1. Define Features (X) and Target (y)
# We can use all activities for this
feature_cols = ['v_mean', 'v_std', 'v_max', 'v_min', 'v_range', 'v_abs_mean', 
                'v_skew', 'v_kurt', 'duration'] # Don't use BMI, as it's derived from weight
target_col = 'weight_kg'

model_data = ml_df[feature_cols + [target_col, 'subject_id']].dropna()

X = model_data[feature_cols]
y = model_data[target_col]

# Scale features
scaler_reg = StandardScaler()
X_scaled = scaler_reg.fit_transform(X)

# 2. Train/Test Split (GroupShuffleSplit)
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X_scaled, y, groups=model_data['subject_id']))

X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

print(f"Data split: {len(y_train)} train, {len(y_test)} test samples.")

# 3. Train Model: Random Forest Regressor
print("\nTraining Random Forest Regressor for Weight...")
rf_weight = RandomForestRegressor(n_estimators=100, random_state=42)
rf_weight.fit(X_train, y_train)
y_pred = rf_weight.predict(X_test)

# 4. Evaluate Regression Model
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Weight Prediction Results:")
print(f"  Mean Absolute Error (MAE): {mae:.2f} kg")
print(f"  R-squared (R2) Score:    {r2:.2f}")

# 5. Plot Results
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.7, edgecolors='k')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.title(f"Weight Prediction (MAE: {mae:.2f} kg)")
plt.xlabel("Actual Weight (kg)")
plt.ylabel("Predicted Weight (kg)")
plt.grid(True, linestyle=':')
plt.show()

## 7. Future Work & Rehabilitation Focus

**Goal:** Analyze feature importance to understand links between signals and muscular weakness.

We can inspect the 'feature importances' from our trained models. For example, which features did the QOM model use most? If it relies heavily on `v_max` and `v_range` during 'jump' signals to identify 'Intervention Priority' cases, this suggests a link between low signal amplitude (weaker force) and poor QOM, which could imply quadriceps weakness.

In [ ]:
print("--- Feature Importance for QOM Model (XGBoost) ---")

# Get feature importances from the trained QOM model
importances = xgb_qom.feature_importances_

# Create a DataFrame for easy plotting
feature_imp_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print(feature_imp_df)

# Plot the importances
plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_imp_df, palette='viridis')
plt.title('Feature Importance for Clinical QOM Classification')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.show()

print("\nInsight: The model's reliance on features like 'v_std' (signal variability) and 'v_range' (peak force)\n")
print("can be analyzed for specific activities (like 'jump') to guide hypotheses about muscular weakness.")